In [10]:
import pandas as pd
import numpy as np
import random
import pickle
import sys
from keras.models import load_model
from keras import backend
import json 

sys.path.append("../../src/models/predict_model")
import predictor
sys.path.append("../../src/data")
import exclusionCriteria
pd.options.mode.chained_assignment = None  # default='warn'

## Get one record for a sample prediction

In [4]:
fileName = 'ED_TOTAL_2010_2010.csv'
with open('../../fileConfig.json') as config_file:    
        fileConfig = json.load(config_file)
fileLocation = fileConfig['dataDirectory'] + fileConfig['processedDirectory'] + fileName
cdc_file = pd.read_csv(fileLocation)    

In [5]:
# records with vital signs = -9 are filtered out for demo purposes
# todo: resolve records with missing data 
reload(exclusionCriteria)
cdc_file = exclusionCriteria.filter_out_records_na(cdc_file)

In [32]:
#getting one record from the input file, as an example
record_index = random.randint(0,len(cdc_file))
sample_record = cdc_file.iloc[[record_index]]
sample_record.head()

,VYEAR,VMONTH,VDAYR,AGE,ARRTIME,WAITTIME,LOV,RESIDNCE,SEX,ETHUN,...,ADMITHOS,OBSHOS,OBSDIS,OTHDISP,ADMIT,HDSTAT,BDATEFL,IMMEDRFL,REGION,MSA
21692,2010,9,7,79,1307,128,338,1,2,2,...,1,0,0,0,6,1,0,0,1,1


## Thresholds and Model Configuration

In [7]:
fileESIThreshold_location = fileConfig['dataDirectory'] + fileConfig['resultDirectory'] + 'esi_thresholds.json'  
with open(fileESIThreshold_location) as config_file:    
        thresholdsConfig = json.load(config_file)
thresholdsConfig

{u'esi1_threshold': 0.8709694147109985,
 u'esi2_threshold': 0.7043555974960327,
 u'rss_threshold': 0.35}

In [8]:
with open('../../modelConfig.json') as config_file:    
        modelConfig = json.load(config_file)
        
modelConfig

{u'co_model': u'../../models/cdc_2009_nn_att_text_embedding.H5',
 u'max_text_length': u'../../models/cdc_2009_att_text_max_length.pickle',
 u'rss_model': u'../../models/cdc_rss_2009_nn_att_text_embedding.H5',
 u'tokenizer': u'../../models/cdc_2009_att_text_tokenizer.pickle'}

## Predict and pull attention relative weights for Model Interpretability

In [33]:
reload(predictor)
att_top = 15
co_pred, pred_rss, pred_esi, co_feature_importance, co_word_heat_map, rss_feature_importance, rss_word_heat_map = \
predictor.predict_and_get_att(modelConfig,thresholdsConfig,  att_top, sample_record)

Creating text for embeddings


In [34]:
print 'y_pred: ' , co_pred
co_feature_importance

y_pred:  [ 0.81400979]


,attention_weight,featureName,input_value
21,0.022847,Severe_Tachypnea,1
27,0.022632,Reason_Shortness_of_Breath,1
49,0.021691,Age_71_80_range,1
1,0.020783,arrival_model,2: NOT by ambulance
41,0.020665,Reason_Musculoskeletal_Other,1
52,0.018637,reason2_for_visit,19201: leg pain ache soreness discomfort
3,0.016431,chf_indicator,0
50,0.014400,Age_81_Above,0
5,0.013086,Age,79
44,0.011641,Age_18_30_range,0


In [35]:
co_word_heat_map

,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8
0,other,s,s,leg,pain,ache,soreness,discomfort
1,0.00803195,0.00644998,0.00644998,0.0185813,0.00895517,0.00790624,0.00832402,0.00974253
2,,,,*,*,,,*
